In [1]:
import pandas as pd


In [6]:

# Load the Excel file
file_path = "Slicer Extensions Download stats as of 2025.03.08.xlsx"  # Update path as needed
excel_data = pd.ExcelFile(file_path)

# Reshape all sheets
reshaped_data = []

for sheet in excel_data.sheet_names:
    # Parse sheet using second row as header (row index 1)
    df = excel_data.parse(sheet, header=1)
    
    # Rename columns for clarity
    df.rename(columns={df.columns[0]: "ID", df.columns[1]: "Extension"}, inplace=True)
    
    # Drop rows without an extension name
    df = df.dropna(subset=["Extension"])
    
    # Add sheet name as date info
    df["Date"] = sheet

    # Reshape the data: each row will be Extension, Version_Info, Downloads
    df_melted = df.melt(id_vars=["Extension", "Date"], 
                        var_name="Version_Info", 
                        value_name="Downloads")

    reshaped_data.append(df_melted)

# Combine all dataframes
long_df = pd.concat(reshaped_data, ignore_index=True)

# Clean up non-numeric download counts
long_df = long_df[pd.to_numeric(long_df["Downloads"], errors="coerce").notnull()]
long_df["Downloads"] = long_df["Downloads"].astype(int)

# Optional: remove rows with invalid version names (e.g., 'ID', 'post-5.x.x')
long_df = long_df[~long_df["Version_Info"].str.contains("post-|ID", na=False)]

# ✅ Remove numeric or bad extension labels
long_df = long_df[long_df["Extension"].apply(lambda x: isinstance(x, str) and not x.strip().isdigit())]


In [7]:

# Group by Extension and Version_Info to get total downloads
grouped = long_df.groupby(["Extension", "Version_Info"]).agg(
    Total_Downloads=("Downloads", "sum")
).reset_index()

# Rank by total downloads
grouped["Rank"] = grouped.groupby("Version_Info")["Total_Downloads"].rank(
    method="dense", ascending=False
)

# Sort results
grouped_sorted = grouped.sort_values(by=["Version_Info", "Rank"])

# Save results if needed
# grouped_sorted.to_csv("slicer_extension_rankings.csv", index=False)

# Display top N (e.g., 10) entries for a quick look
print(grouped_sorted.head(10))


                       Extension Version_Info  Total_Downloads  Rank
0                            ABC        4.0.0                0   1.0
63   ABLTemporalBoneSegmentation        4.0.0                0   1.0
126              AblationPlanner        4.0.0                0   1.0
189           AirwaySegmentation        4.0.0                0   1.0
252         AnglePlanesExtension        4.0.0                0   1.0
315    AnomalousFiltersExtension        4.0.0                0   1.0
378            ArduinoController        4.0.0                0   1.0
441              AstmPhantomTest        4.0.0                0   1.0
504                     Auto3dgm        4.0.0                0   1.0
601                   Autoscroll        4.0.0                0   1.0


In [9]:
# Group by Extension only (not by version)
extension_totals = long_df.groupby("Extension").agg(
    Total_Downloads=("Downloads", "sum")
).reset_index()


# Rank extensions by total downloads
extension_totals["Rank"] = extension_totals["Total_Downloads"].rank(
    method="dense", ascending=False
)

# Sort by rank
ranked_extensions = extension_totals.sort_values(by="Rank")

# Save or display
# ranked_extensions.to_csv("slicer_extensions_ranked_by_downloads.csv", index=False)
print(ranked_extensions.head(25))

                     Extension  Total_Downloads  Rank
274            UKFTractography           467600   1.0
243                 SlicerVMTK           464705   2.0
285          WindowLevelEffect           433483   3.0
35                  CurveMaker           431953   4.0
256         SwissSkullStripper           426138   5.0
171         SegmentationWizard           403562   6.0
135      PickAndPaintExtension           401825   7.0
282                 VolumeClip           398399   8.0
157                 SPHARM-PDM           385916   9.0
262                     TOMAAT           381251  10.0
260                TCIABrowser           379101  11.0
216        SlicerLayoutButtons           362576  12.0
257                  T1Mapping           359938  13.0
259                  T2mapping           358297  14.0
294           lapdMouseBrowser           353581  15.0
156             SNRMeasurement           347564  16.0
286                 XNATSlicer           343119  17.0
73                       IAS

In [14]:
# ✅ Extract and filter only versions >= 5.8.0
import re
from packaging.version import Version


def extract_version(v):
    match = re.match(r"(\d+\.\d+(\.\d+)?)", str(v))
    return Version(match.group(1)) if match else None

long_df["Parsed_Version"] = long_df["Version_Info"].apply(extract_version)
long_df = long_df[long_df["Parsed_Version"].notnull()]
long_df = long_df[long_df["Parsed_Version"] >= Version("5.8.0")]

# Group and rank by extension
filtered_rank = long_df.groupby("Extension").agg(
    Total_Downloads=("Downloads", "sum")
).reset_index()

filtered_rank["Rank"] = filtered_rank["Total_Downloads"].rank(method="dense", ascending=False)
filtered_rank = filtered_rank.sort_values(by="Rank")

# Save or preview
# filtered_rank.to_csv("slicer_extensions_ranked_5.8_and_later.csv", index=False)
print(filtered_rank.head(20))

                     Extension  Total_Downloads  Rank
142                    PyTorch             4528   1.0
111                     NNUNet             3015   2.0
235     SlicerPythonTestRunner             2943   3.0
99              MarkupsToModel             2787   4.0
162  SegmentEditorExtraEffects             2667   5.0
267           TotalSegmentator             2299   6.0
236                   SlicerRT             1281   7.0
36                       DCMQI             1201   8.0
51           DentalSegmentator             1098   9.0
205              SlicerElastix             1090  10.0
94              MONAIAuto3DSeg             1071  11.0
255        SurfaceWrapSolidify             1011  12.0
222                SlicerMorph              946  13.0
211                  SlicerIGT              939  14.0
243                 SlicerVMTK              934  15.0
254              SurfaceMarkup              924  16.0
203   SlicerDevelopmentToolbox              920  17.0
210                SlicerIGS